# Anders' modeling notebook: Reproducible data science with amie
Anders now uses scikit learn to build a model that predicts the wine quality based on the chemical features that Fritz measured. This takes some experimentation: He tries different models and optimizes their parameters. Amie automatically tracks the model parameters and their performance along with the training and testing data. Even if this notebook is lost, he can come back re-create the models, and develop them further.


In [ ]:
import amieci
import random
import io
import base64
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

def train_and_eval(garden, model, train_x, test_x, train_y, test_y, parent=None, save=True):
    
    model.fit(train_x, train_y)
    pred = model.predict(test_x)
    
    modelname = str(model).split('(')[0]   
    model_params = model.get_params()
    
    leaf = garden.ct.new_leaf(parent=parent)
    leaf.set_title(modelname)
    leaf.set_description(modelname)
    leaf.kvs.add('model_name', modelname)
    leaf.kvs.load_from_dict(model_params)
    leaf.kvs.add('mae', mae(test_y, pred))
    
    if save:
        leaf.save()
    
    return leaf, model

def mae(y, pred):
    return np.mean(abs(y-pred))


# Log in on amie.ai and go to "www.amie.ai/#/user to get your API key
garden = amieci.Garden(key="anders_key")
garden.load()


## The right branch
He sets the current leaf to where he left off with the basic data analysis, so he can continue working in this branch

In [ ]:
garden.set_cl('ea1f4d24-e4fe-4806-8fac-4afcbf2e19eb')

## Load the data again
He now uses the same script as in the analysis notebook to load the data from Fritz and Francesca.

In [ ]:
#Load the data again
tree_ids = ['0a5fd360-038f-4cd8-9294-069dea4fad66', 'f6dc0e23-81dd-4a87-84d8-bfb158025a79']
tips = garden.tips()
dataframes = []
for tree in tree_ids:
    for data_leaf in tips[tree]:
        #I take the first (and only) file, because I can see that on the graph
        try:
            datafile = garden.leaves[data_leaf].download_data(filename='.csv')[0]
            dataframes.append(pd.read_csv(datafile, sep=';',index_col=0))
        except:
            pass
#Concatenate the data
data = pd.concat(dataframes, axis=1)

## Prepare for modeling and store the data
To make everything reproducible, he also adds the test train split and the normalization to the leaf.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer, MinMaxScaler

try:
    data_leaf = garden.ct.new_leaf()
except:
    pass

train, test = train_test_split(data)

garden.ct.cl.kvs.add("test_train_split", "0.25,0.75")

train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(train_x)
train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)
train_y = np.ravel(train_y)
test_y = np.ravel(test_y)

save_train_x = bytes(pd.DataFrame(train_x).to_csv(), 'utf-8')
save_test_x = bytes(pd.DataFrame(test_x).to_csv(), 'utf-8')
save_train_y = bytes(pd.DataFrame(train_y).to_csv(), 'utf-8')
save_test_y = bytes(pd.DataFrame(test_y).to_csv(), 'utf-8')
data_leaf.add_data('train_x.csv',save_train_x, caption='the split off training features')
data_leaf.add_data('test_x.csv',save_test_x, caption='the split off testing features')
data_leaf.add_data('train_y.csv',save_train_y, caption='the split off training labels')
data_leaf.add_data('test_y',save_test_y, caption='the split off testing labels')

In [ ]:
garden.ct.cl.set_title("Test-train split, and normalization")
garden.ct.cl.set_description("""The predicted column is quality; 
                             a scalar from [3,8]. All features normalized to [0,1]""")
garden.ct.cl.save()

## Modeling
He now runs his models.

In [ ]:
# Machine Learning Models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
model_leaf = garden.ct.new_leaf()
model_leaf.set_title("The modeling plan")
model_leaf.set_description("""Models we want to try: \n \
Linear Regression \n \ 
Support Vector Machine Regression \n \ 
Random Forest Regression \n \
Gradient Boosting Regression \n \
K-Nearest Neighbors Regression \n """)

baseline = np.median(train_y)

model_leaf.append_description("Average is used as the baseline")
model_leaf.kvs.add('baseline_mae', mae(test_y, baseline))
model_leaf.save()

### Branching
Since there might be further optimization steps for each model, Anders makes a new branch for each model by attaching the new leaf to the same leaf via ```parent = model_leaf``` which was defined above.

In [ ]:
lr_leaf, lr = train_and_eval(garden, 
                             LinearRegression(), 
                             train_x, test_x, train_y, test_y, 
                             parent=model_leaf)
rf_leaf, rf = train_and_eval(garden, RandomForestRegressor(), 
                             train_x, test_x, train_y, test_y, 
                             parent=model_leaf)
svr_leaf, svr = train_and_eval(garden, 
                               SVR(), 
                               train_x, test_x, train_y, test_y, 
                               parent=model_leaf)
gb_leaf, gb = train_and_eval(garden, 
                             GradientBoostingRegressor(), 
                             train_x, test_x, train_y, test_y, 
                             parent=model_leaf)
knn_leaf, knn = train_and_eval(garden, 
                               KNeighborsRegressor(), 
                               train_x, test_x, train_y, test_y, 
                               parent=model_leaf)

# Hyperparameter optimization
Anders can quickly compare the models in the app by plotting mae over model name. He chooses to go further with 3 of them and optimizes their hyperparameters. Amie tracks all parameters in the optimization step. Since this might take a while, Anders can check through the app, from home or on his phone, how are things are. Afterwards he can analyse model parameters and results by plotting them in the app.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [ ]:
grid = {'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'gamma' : [0.01, 0.1, 1, 10, 100]}
grid_search = GridSearchCV(svr, grid)
grid_search.fit(train_x, train_y)
svr_leaf, svr = train_and_eval(garden, grid_search.best_estimator_,
                               train_x, test_x, train_y, test_y, 
                               parent=svr_leaf, save=False)
svr_leaf.kvs.add('grid', grid)
svr_leaf.append_description("Hyperparameter optimization with gridsearch")
svr_leaf.set_title("Optimization SVR")
svr_leaf.save()

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 2)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 2)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(train_x, np.ravel(train_y))
rf_leaf, rf = train_and_eval(garden, rf_random.best_estimator_,
                             train_x, test_x, train_y, test_y, 
                             parent=rf_leaf, 
                             save=False)
rf_leaf.kvs.add('grid', random_grid)
rf_leaf.append_description("Hyperparameter optimization with a random grid")
rf_leaf.set_title("Optimization Random Forest")
rf_leaf.save()






In [ ]:
loss = ['ls', 'lad', 'huber']
n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
min_samples_leaf = [1, 2, 4, 6, 8]
min_samples_split = [2, 4, 6, 10]
max_features = ['auto', 'sqrt', 'log2', None]
random_grid = {'loss': loss,
                       'n_estimators': n_estimators,
                       'max_depth': max_depth,
                       'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split,
                       'max_features': max_features}
gb_random = RandomizedSearchCV(estimator=gb,
                               param_distributions=random_grid,
                               cv=4, n_iter=25, 
                               scoring = 'neg_mean_absolute_error',
                               n_jobs = -1, verbose = 1, 
                               return_train_score = True,
                               random_state=42)
gb_random.fit(test_x, test_y)
gb_leaf, gb = train_and_eval(garden, gb_random.best_estimator_,
                               train_x, test_x, train_y, test_y, 
                               parent=gb_leaf,
                               save=False)
gb_leaf.kvs.add('grid', random_grid)
gb_leaf.append_description("Hyperparameter optimization with a random grid")
gb_leaf.set_title("Optimization Gradient Boosting Regression")
gb_leaf.save()


# Find the winner with graphiera and save the model

A quick plot of mae over model_name in the app reveals the winner. Anders now saves the model to a new leaf and adds some visualizations.

In [ ]:
rf.fit(train_x, train_y)
residuals = rf.predict(test_x) - test_y

# Plot the residuals in a histogram
fig_res = plt.figure()
plt.hist(residuals, bins = 25,
         edgecolor = 'black')
plt.xlabel('Error'); plt.ylabel('Count')
plt.title('Distribution of Residuals');
plt.show()

In [ ]:
df = pd.DataFrame(rf.feature_importances_).transpose()
df = pd.DataFrame(data = rf.feature_importances_.reshape([1,11]), columns=data.columns.tolist()[0:-1], index=['importance'])
df_array = np.array(df)[0]
y = np.arange(1,12)
plt.style.use('fivethirtyeight')
fig_feat = plt.figure()
plt.barh(y,df_array,height=0.5)
plt.yticks(y,list(df.columns))
plt.title('Feature Importances', fontweight='bold')

In [ ]:
import io
import base64
rf = RandomForestRegressor(random_state=60)
from sklearn.externals import joblib
file = io.BytesIO()
joblib.dump(rf, file) 
file.flush()
winning_model = base64.b64encode(file.getvalue())

In [ ]:
garden.ct.new_leaf(parent = rf_leaf)
garden.ct.cl.set_title("Winning model")
garden.ct.cl.set_description("""The best model was the #model_name RandomForestRegressor#. The model is stored in a pickle $File 1$. It can be downloaded and re-run with scikit-learn.
Residuals look fine (see $Img 2$), they are well centered around 0. With this relatively quick process and some re-measuring of the data, we could get to #mae 0.399#, which is quite OK, since the quality is given in increments of 1. 
The models improved quite a bit with optimization, plot #model_name # over #mae # to see the result. Some more work on this should get us even closer.
More more analysis and visualization of the forest should also reveal the non-linear correlations between the features, but let's first see what we can control. 
The most important features are sulfates and alcohol content, see $Img 3$.
""")
garden.ct.cl.add_data('rf.pkl', winning_model, caption="the winning model")
garden.ct.cl.add_plot('residuals.png',fig_res,caption='histogram over the residuals')
garden.ct.cl.add_plot('features.png',fig_feat,caption='histogram over the residuals')
garden.ct.cl.save()